# Tổng hợp chỉ số RAGAS: eval_proposed vs eval_baseline

Notebook này giúp bạn:
1. **Tải lên 2 file CSV**: `eval_proposed` và `eval_baseline`
2. Tính trung bình các chỉ số: **context_recall, answer_correctness, faithfulness, answer_relevancy**
3. Xuất bảng tổng hợp gồm **3 dòng**:
   - Trung bình toàn bộ `eval_proposed`
   - Trung bình toàn bộ `eval_baseline`
   - Trung bình trên **các câu hỏi xuất hiện ở cả 2 file** (khớp theo `user_input`) — tính bằng cách **lấy trung bình (proposed+baseline)/2 theo từng câu**, rồi lấy trung bình tiếp theo toàn bộ tập giao.

> Gợi ý: Nếu bạn chạy trên **Google Colab**, cell upload sẽ hiện nút chọn file.


In [ ]:
# ==== 1) Import ====
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", 120)

METRICS = ["context_recall", "answer_correctness", "faithfulness", "answer_relevancy"]

def _normalize_user_input(x: str) -> str:
    """Chuẩn hoá để khớp user_input giữa 2 file (giảm sai khác do khoảng trắng/newline)."""
    if pd.isna(x):
        return ""
    x = str(x)
    x = x.replace("\u00a0", " ")  # non-breaking space
    x = re.sub(r"\s+", " ", x).strip()
    return x

def _is_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False


In [ ]:
# ==== 2) Upload / chọn đường dẫn file ====
# Nếu chạy trên Colab: upload 2 file CSV (eval_proposed và eval_baseline).
# Nếu chạy trên Jupyter local: đặt proposed_path / baseline_path trỏ tới file CSV của bạn.

uploaded_files = None
if _is_colab():
    from google.colab import files  # type: ignore
    uploaded_files = files.upload()
    print("Uploaded:", list(uploaded_files.keys()))
else:
    print("Không phải Colab. Hãy đặt biến proposed_path / baseline_path ở cell bên dưới.")

# Nếu bạn muốn tự đặt tên file, chỉnh ở cell tiếp theo.


In [ ]:
# ==== 3) Set đường dẫn file ====
# Cách 1 (Colab): Nếu bạn upload đúng 2 file, notebook sẽ cố gắng tự nhận diện.
# Cách 2: Tự gán proposed_path và baseline_path theo tên file/đường dẫn.

proposed_path = None
baseline_path = None

if uploaded_files is not None:
    names = list(uploaded_files.keys())
    # Heuristic: ưu tiên file có chữ "proposed"/"baseline" trong tên
    proposed_candidates = [n for n in names if "proposed" in n.lower()]
    baseline_candidates = [n for n in names if "baseline" in n.lower() or "ragas" in n.lower()]

    if proposed_candidates:
        proposed_path = proposed_candidates[0]
    if baseline_candidates:
        baseline_path = baseline_candidates[0]

    # fallback: nếu không nhận diện được, lấy theo thứ tự upload
    if proposed_path is None and len(names) >= 1:
        proposed_path = names[0]
    if baseline_path is None and len(names) >= 2:
        baseline_path = names[1]

# ---- Nếu bạn chạy local hoặc muốn override, bỏ comment và sửa 2 dòng dưới ----
# proposed_path = "eval_proposed.csv"
# baseline_path = "eval_baseline.csv"

assert proposed_path is not None and baseline_path is not None, (
    "Chưa có proposed_path/baseline_path. Hãy gán thủ công ở cuối cell này."
)

print("proposed_path =", proposed_path)
print("baseline_path =", baseline_path)


In [ ]:
# ==== 4) Load dữ liệu ====
df_proposed = pd.read_csv(proposed_path)
df_baseline = pd.read_csv(baseline_path)

print("df_proposed shape:", df_proposed.shape)
print("df_baseline shape:", df_baseline.shape)

display(df_proposed.head(3))
display(df_baseline.head(3))


In [ ]:
# ==== 5) Validate cột & ép kiểu số ====
required_cols = set(["user_input"] + METRICS)

missing_p = required_cols - set(df_proposed.columns)
missing_b = required_cols - set(df_baseline.columns)
assert not missing_p, f"eval_proposed thiếu cột: {sorted(missing_p)}"
assert not missing_b, f"eval_baseline thiếu cột: {sorted(missing_b)}"

for m in METRICS:
    df_proposed[m] = pd.to_numeric(df_proposed[m], errors="coerce")
    df_baseline[m] = pd.to_numeric(df_baseline[m], errors="coerce")

# key để khớp câu hỏi
df_proposed["_key"] = df_proposed["user_input"].map(_normalize_user_input)
df_baseline["_key"] = df_baseline["user_input"].map(_normalize_user_input)

# loại bỏ key rỗng
df_proposed = df_proposed[df_proposed["_key"] != ""].copy()
df_baseline = df_baseline[df_baseline["_key"] != ""].copy()

print("After cleaning empty user_input:")
print("df_proposed shape:", df_proposed.shape)
print("df_baseline shape:", df_baseline.shape)


In [ ]:
# ==== 6) Tính trung bình (3 dòng) ====
# (A) Trung bình toàn bộ proposed
proposed_mean = df_proposed[METRICS].mean(numeric_only=True)

# (B) Trung bình toàn bộ baseline
baseline_mean = df_baseline[METRICS].mean(numeric_only=True)

# (C) Trung bình trên các câu chung (inner join theo _key)
# Nếu có duplicate _key, sẽ giữ tất cả cặp join -> có thể làm lệch.
# Để ổn định, ta sẽ:
#   1) groupby _key và lấy mean metrics theo từng key cho mỗi file
#   2) inner join theo _key
#   3) lấy (proposed + baseline)/2 theo từng key, rồi mean toàn bộ

p_g = df_proposed.groupby("_key")[METRICS].mean().reset_index()
b_g = df_baseline.groupby("_key")[METRICS].mean().reset_index()

common = p_g.merge(b_g, on="_key", suffixes=("_proposed", "_baseline"), how="inner")
n_common = common.shape[0]

if n_common == 0:
    common_avg = pd.Series({m: np.nan for m in METRICS})
else:
    per_question_avg = pd.DataFrame({
        m: (common[f"{m}_proposed"] + common[f"{m}_baseline"]) / 2.0
        for m in METRICS
    })
    common_avg = per_question_avg.mean()

summary = pd.DataFrame(
    [proposed_mean, baseline_mean, common_avg],
    index=["mean_eval_proposed", "mean_eval_baseline", "mean_common_questions_(avg_of_two)"],
)

# Thêm cột đếm để dễ kiểm tra
summary.insert(0, "n_rows", [len(df_proposed), len(df_baseline), n_common])

summary


In [ ]:
# ==== 7) Xuất kết quả ====
out_csv = "ragas_summary.csv"
summary.to_csv(out_csv, index=True, encoding="utf-8-sig")
print("Saved:", out_csv)

# Nếu muốn xem danh sách các câu chung:
common_keys = common[["_key"]].copy()
common_keys.columns = ["user_input_key_normalized"]
display(common_keys.head(10))
print("Total common questions:", n_common)
